In [1]:
import tensorflow as tf
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    explained_variance_score
)
import matplotlib.pyplot as plt

# Ensure TensorFlow uses GPU
if tf.config.list_physical_devices('GPU'):
    print("GPU is available, setting memory growth...")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU found, using CPU instead.")


GPU is available, setting memory growth...


In [3]:
# Configuración
MODEL_PATH = "tourism_recommendation_dnn.h5"
TEST_FILE = "test3.parquet"
BATCH_SIZE = 2048 

def load_and_predict():
    model = tf.keras.models.load_model(MODEL_PATH)
    
    all_targets = []
    all_preds = []
    
    parquet_file = pq.ParquetFile(TEST_FILE)
    total_batches = parquet_file.metadata.num_row_groups
    processed_samples = 0
    
    for batch_idx, batch in enumerate(parquet_file.iter_batches(batch_size=BATCH_SIZE)):
        features = np.array([
            np.array(f.as_py(), dtype=np.float32) 
            for f in batch['features']
        ], dtype=np.float32)
        
        targets = batch['target'].to_numpy().astype(np.float32)
        
        # Predicciones
        preds = model.predict(features, verbose=0).flatten()
        
        all_targets.extend(targets)
        all_preds.extend(preds)
        processed_samples += len(targets)
        
        if (batch_idx + 1) % 10 == 0:
            print(f"Processed {batch_idx+1}/{total_batches} batches "
                  f"({processed_samples} samples)")

    targets = np.array(all_targets)
    preds = np.array(all_preds)
    
    return targets, preds

def calculate_metrics(targets, preds):
    mae = mean_absolute_error(targets, preds)
    mse = mean_squared_error(targets, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(targets, preds)
    evs = explained_variance_score(targets, preds)

    errors = preds - targets
    std_dev = np.std(errors)
    error_stats = {
        'within_0.5': np.mean(np.abs(errors) <= 0.5),
        'within_1.0': np.mean(np.abs(errors) <= 1.0),
        'within_1.5': np.mean(np.abs(errors) <= 1.5),
    }

    print("\n" + "="*60)
    print("Model Evaluation Metrics:")
    print(f"- MAE (Mean Absolute Error): {mae:.3f} stars")
    print(f"- RMSE (Root Mean Squared Error): {rmse:.3f} stars")
    print(f"- R² Score: {r2:.3f}")
    print(f"- Explained Variance Score: {evs:.3f}")
    print(f"- Error Standard Deviation: {std_dev:.3f}")
    print("\nError Distribution:")
    print(f"- Predictions within ±0.5 stars: {error_stats['within_0.5']:.1%}")
    print(f"- Predictions within ±1.0 stars: {error_stats['within_1.0']:.1%}")
    print(f"- Predictions within ±1.5 stars: {error_stats['within_1.5']:.1%}")
    print("="*60)

    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.scatter(targets, preds, alpha=0.3)
    plt.plot([0, 5], [0, 5], '--r')
    plt.xlabel('Actual Ratings')
    plt.ylabel('Predicted Ratings')
    plt.title('Actual vs Predicted Ratings')
    
    plt.subplot(1, 2, 2)
    plt.hist(errors, bins=50, density=True)
    plt.xlabel('Prediction Error (Predicted - Actual)')
    plt.ylabel('Density')
    plt.title('Error Distribution')
    
    plt.tight_layout()
    plt.show()

targets, preds = load_and_predict()
calculate_metrics(targets, preds)

Processed 10/17 batches (20480 samples)
Processed 20/17 batches (40960 samples)
Processed 30/17 batches (61440 samples)
Processed 40/17 batches (81920 samples)
Processed 50/17 batches (102400 samples)
Processed 60/17 batches (122880 samples)
Processed 70/17 batches (143360 samples)
Processed 80/17 batches (163840 samples)
Processed 90/17 batches (184320 samples)
Processed 100/17 batches (204800 samples)
Processed 110/17 batches (225280 samples)
Processed 120/17 batches (245760 samples)
Processed 130/17 batches (266240 samples)
Processed 140/17 batches (286720 samples)
Processed 150/17 batches (307200 samples)
Processed 160/17 batches (327680 samples)
Processed 170/17 batches (348160 samples)
Processed 180/17 batches (368640 samples)
Processed 190/17 batches (389120 samples)
Processed 200/17 batches (409600 samples)
Processed 210/17 batches (430080 samples)
Processed 220/17 batches (450560 samples)
Processed 230/17 batches (471040 samples)
Processed 240/17 batches (491520 samples)
Proce

In [10]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

def plot_regression_confusion_matrix(targets, preds, rating_range=(1,5)):
    actual_ratings = np.clip(np.round(targets), rating_range[0], rating_range[1]).astype(int)
    predicted_ratings = np.clip(np.round(preds), rating_range[0], rating_range[1]).astype(int)
    
    cm = confusion_matrix(actual_ratings, predicted_ratings, 
                         labels=np.arange(rating_range[0], rating_range[1]+1))
    
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    plt.figure(figsize=(10,8))
    sns.heatmap(cm_percent, annot=True, fmt=".1f", cmap="Blues",
                xticklabels=np.arange(rating_range[0], rating_range[1]+1),
                yticklabels=np.arange(rating_range[0], rating_range[1]+1),
                cbar_kws={'label': 'Percentage of Predictions'})
    
    plt.title("Confusion Matrix (Percent of Predictions by True Rating)")
    plt.xlabel('Predicted Rating')
    plt.ylabel('True Rating')
    plt.show()

    print("\nClassification Metrics for Discretized Ratings:")
    for rating in np.unique(actual_ratings):
        mask = actual_ratings == rating
        precision = np.mean(predicted_ratings[mask] == rating)
        recall = np.mean(actual_ratings[predicted_ratings == rating] == rating)
        print(f"Rating {rating}:")
        print(f"  Precision: {precision:.1%} | Recall: {recall:.1%}")

plot_regression_confusion_matrix(targets, preds)


Classification Metrics for Discretized Ratings:
Rating 1:
  Precision: 0.1% | Recall: 76.6%
Rating 2:
  Precision: 1.4% | Recall: 12.7%
Rating 3:
  Precision: 23.4% | Recall: 16.3%
Rating 4:
  Precision: 75.8% | Recall: 28.0%
Rating 5:
  Precision: 13.1% | Recall: 64.2%
